In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/Shareddrives/Unlimited/Colab/AlphaVision/"
MODEL = "aug"
model_path = "models_" + MODEL + "/"

In [ ]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/Shareddrives/Unlimited/Colab/AlphaVision/AlphaVision.zip" "data_2.zip"
# data_2.zip을 현재 디렉터리에 압축해제
!unzip "data_2.zip"

Archive:  data_2.zip
  inflating: dirty_mnist_2nd.zip     
  inflating: dirty_mnist_2nd_answer.csv  
  inflating: mnist_data.zip          
  inflating: sample_submission.csv   
  inflating: test_dirty_mnist_2nd.zip  


In [ ]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
#from torchinfo import summary

from torchvision import transforms
from torchvision.models import densenet121

## 1. 커스텀 데이터셋 만들기

In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

## 2. 이미지 어그멘테이션

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [ ]:
trainset = MnistDataset('dirty_mnist', 'dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('test_dirty_mnist', 'sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=64, num_workers=8)
test_loader = DataLoader(testset, batch_size=32, num_workers=4)

## 3. ResNet50 모형

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Tue Feb 23 07:56:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = densenet121(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
#print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


## 4. 학습하기

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 10
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.68887, 0.55168
0: 0.68350, 0.54928
0: 0.67821, 0.54627
0: 0.67247, 0.55950
0: 0.65342, 0.58113
0: 0.64321, 0.58714
0: 0.63559, 0.59255
1: 0.60525, 0.63281
1: 0.62939, 0.62139
1: 0.60443, 0.63642
1: 0.60803, 0.62620
1: 0.60395, 0.64904
1: 0.58468, 0.66767
1: 0.57902, 0.66226
2: 0.55665, 0.69291
2: 0.58030, 0.67488
2: 0.54451, 0.69952
2: 0.55285, 0.69171
2: 0.54228, 0.70553
2: 0.52214, 0.72236
2: 0.53474, 0.72055
3: 0.47694, 0.76322
3: 0.50334, 0.74700
3: 0.48008, 0.76502
3: 0.48618, 0.75421
3: 0.47683, 0.76683
3: 0.48655, 0.75300
3: 0.46939, 0.76262
4: 0.41022, 0.80769
4: 0.45689, 0.77764
4: 0.41613, 0.80649
4: 0.44623, 0.78005
4: 0.40556, 0.82632
4: 0.43703, 0.78425
4: 0.43788, 0.79267
5: 0.39016, 0.82272
5: 0.40635, 0.81250
5: 0.38055, 0.83293
5: 0.41728, 0.81190
5: 0.37540, 0.82993
5: 0.40463, 0.80829
5: 0.38441, 0.82752
6: 0.34114, 0.84675
6: 0.37528, 0.83654
6: 0.36297, 0.84796
6: 0.36351, 0.85397
6: 0.35414, 0.84856
6: 0.36533, 0.83654
6: 0.36317, 0.83534
7: 0.31328, 0.87079


## 5. 추론하기

In [ ]:
submit = pd.read_csv('sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv(path + model_path + "baseline_prediction.csv", index = False)